In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc")

SLG [I]: Checking config correctness... Config correct!


100%|██████████| 3/3 [00:00<00:00, 23.13it/s]

SLG [I]: Dataset loaded from disk (TXT files)
SLG [I]: Vocabulary loaded from disk


SLG [I]: nGrams loaded from disk (['6.json', '7.json', '10.json', '2.json', '3.json', '8.json', '4.json', '5.json', '9.json'])
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-26 11:26:38.101962: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-26 11:26:38.105585: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


SLG [I]: Paradigmatizer loaded from disk


In [25]:
from datasets import Dataset
from tqdm.notebook import tqdm

corpus = semiotic.corpus.train

corpus_norm = []
for sent in tqdm(corpus["text"]):
    corpus_norm.append(semiotic.syntagmatic.tokenizer.normalizer.normalize_str(sent))

corpus_norm = Dataset.from_dict({"text":corpus_norm})

  0%|          | 0/5423648 [00:00<?, ?it/s]

In [27]:
# ng4 = slg.vocabulary.nGram.build(corpus=corpus,n=4, str_normalizer=semiotic.syntagmatic.tokenizer.normalizer.normalize_str, parallel=False, keep_in_memory=True)

ng_n = 10
ng = slg.vocabulary.nGram.build(
    corpus=corpus_norm,
    n=ng_n,
    parallel=False,
    keep_in_memory=True
    )
slg.util.save_file(ng,semiotic.paths.vocabulary / f"ngrams/{ng_n}.json")

SLG [W]: This feature is not fully implemented/tested yet
SLG: Counting 10-Grams...


  0%|          | 0/413051903 [00:00<?, ?it/s]

In [3]:
from collections import defaultdict
import seaborn as sns
import networkx as nx
import graphviz as gv
from ipywidgets import interact
from math import log
from numpy.linalg import svd
import numpy as np
from itertools import chain, combinations
from tqdm.notebook import tqdm

In [4]:
ng1 = semiotic.vocab.ng1
ng2 = semiotic.vocab.ng2
ng3 = semiotic.vocab.ng3
ng4 = semiotic.vocab.ng4
ng5 = semiotic.vocab.ng5
ng6 = semiotic.vocab.ng6
ng7 = semiotic.vocab.ng7
ng8 = semiotic.vocab.ng8
ng9 = semiotic.vocab.ng9
ng10 = semiotic.vocab.ng10

In [5]:
norm_factor = 1/sum(ng1.freq.values())

ng1.prob = {k:v*norm_factor for k,v in ng1.freq.items()}
ng2.prob = {k:v*norm_factor for k,v in ng2.freq.items()}
ng3.prob = {k:v*norm_factor for k,v in ng3.freq.items()}
ng4.prob = {k:v*norm_factor for k,v in ng4.freq.items()}
ng5.prob = {k:v*norm_factor for k,v in ng5.freq.items()}
ng6.prob = {k:v*norm_factor for k,v in ng6.freq.items()}
ng7.prob = {k:v*norm_factor for k,v in ng7.freq.items()}
ng7.prob = {k:v*norm_factor for k,v in ng7.freq.items()}
ng8.prob = {k:v*norm_factor for k,v in ng8.freq.items()}
ng9.prob = {k:v*norm_factor for k,v in ng9.freq.items()}
ng10.prob = {k:v*norm_factor for k,v in ng10.freq.items()}

# ng_prob = {**ng1.prob,**ng2.prob,**ng3.prob, **ng4.prob, **ng5.prob, **ng6.prob, **ng7.prob}
ng_prob = [ng1.prob, ng2.prob, ng3.prob, ng4.prob, ng5.prob, ng6.prob, ng7.prob, ng8.prob, ng9.prob, ng10.prob]

In [6]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [11]:
from random import randint
sent = semiotic.corpus.train[randint(0,100000)]["text"]
sent = "I have made my plans"
# sent = "verygood"
print(sent)
sent = semiotic.syntagmatic.tokenizer.normalizer.normalize_str(sent)



I have made my plans


In [12]:
len_sent = len(sent)
pws = list(powerset(range(len_sent+1)))
possible_cuts = [i for i in pws if 0 in i and len_sent in i]
possible_offsets = []
for cut in tqdm(possible_cuts):
    possible_offsets.append(list(zip(cut,cut[1:])))
possible_offsets_n = []
for offset in possible_offsets:
    if max([r-l for l,r in offset])<9 and offset != [(0,len(sent))]:
        possible_offsets_n.append(offset)
seg_probs = []
for offsets in tqdm(possible_offsets_n):
    segs = [sent[l:r] for l,r in offsets]
    seg_probs.append((np.prod([ng_prob[len(seg)-1][seg] for seg in segs]), segs))

sorted(seg_probs, reverse=True)[:10]

  0%|          | 0/32768 [00:00<?, ?it/s]

  0%|          | 0/32192 [00:00<?, ?it/s]

[(1.0052526847183316e-13, ['i', 'havemade', 'myplans']),
 (2.3299475427595537e-14, ['i', 'havemade', 'm', 'yplans']),
 (2.1298318664759638e-14, ['i', 'havemade', 'myplan', 's']),
 (1.2581321579931439e-14, ['i', 'havemad', 'emyplans']),
 (1.236343015965356e-14, ['i', 'havemad', 'e', 'myplans']),
 (1.0995390151505617e-14, ['ihavemad', 'emyplans']),
 (1.0804964911883531e-14, ['ihavemad', 'e', 'myplans']),
 (9.35750897157668e-15, ['i', 'havemade', 'm', 'yplan', 's']),
 (7.840117907565132e-15, ['i', 'havemade', 'my', 'plans']),
 (7.005707550132993e-15, ['i', 'havemade', 'm', 'y', 'plans'])]

In [9]:
from collections import Counter
bla = Counter(ng_prob)

In [13]:
bla.most_common(10)

[('e', 0.12283516959154912),
 ('t', 0.09144317330136345),
 ('a', 0.07986945016799842),
 ('o', 0.07554999657567639),
 ('i', 0.07229377970004593),
 ('n', 0.07001552324870404),
 ('s', 0.06436566038490976),
 ('r', 0.0612647297217823),
 ('h', 0.05187486568481037),
 ('l', 0.04127051180816118)]